In [ ]:
# import libraries
import osmnx as ox
import networkx as nx
import pandas as pd
import ast
import matplotlib.pyplot as plt
import concurrent.futures
from shapely.geometry import LineString
import geopandas as gpd

In [ ]:
G = ox.io.load_graphml('../Data/LA_clip_convex_strong_network.graphml')

In [ ]:
# import routes
route1 = pd.read_csv('../Data/model_iteration/route_result_uber_hod3am_10000_freeflow.csv')
route2 = pd.read_csv('../Data/model_iteration/route_result_uber_hod3am_10000_bo_model1.csv')
route3 = pd.read_csv('../Data/model_iteration/route_result_uber_hod3am_10000_bo_model2.csv')

In [ ]:
remove = [17, 840, 2179, 3795, 4380, 5747, 8436, 8517]
route1 = route1.drop(remove).reset_index(drop=True)
route2 = route2.drop(remove).reset_index(drop=True)
route3 = route3.drop(remove).reset_index(drop=True)

In [ ]:
route1['freeflow_route'] = [ast.literal_eval(route) for route in route1['freeflow_route']]
route2['penalized_route'] = [ast.literal_eval(route) for route in route2['penalized_route']]
route3['penalized_route'] = [ast.literal_eval(route) for route in route3['penalized_route']]

In [ ]:
# import google routes
od_data = pd.read_csv('../Data/routesdataadded10000.csv')

In [ ]:
od_data.columns

In [ ]:
for node, attr in G.nodes(data=True):
    G.nodes[node]['y'] = float(attr['y'])
    G.nodes[node]['x'] = float(attr['x'])

In [ ]:
## find top 20 routes that are different in travel time
diff = pd.DataFrame()
diff['route1'] = od_data['duration'] - route1['freeflow_travel_time']
diff['route2'] = od_data['duration'] - route2['penalized_total_time']
diff['route3'] = od_data['duration'] - route3['penalized_total_time']
diff['total'] = diff['route1'] + diff['route2'] + diff['route3']

In [ ]:
diff = diff.dropna()
diff

In [ ]:
diff.columns

In [ ]:
timecomparison = pd.DataFrame()
timecomparison['route1'] = route1['freeflow_travel_time']
timecomparison['route2'] = route2['penalized_total_time']
timecomparison['route3'] = route3['penalized_total_time']
timecomparison['google'] = od_data['duration']

In [ ]:
timecomparison

## Greatest

In [ ]:
diff = diff.sort_values(ascending=False, by='total')
diff.head(20)

In [ ]:
most_diff = diff.head(20).index
most_diff

In [ ]:
od_data_longer = od_data['polyline'][most_diff]

In [ ]:
od_data_longer

In [ ]:
linestrings = [LineString(ast.literal_eval(item)) for item in od_data_longer]

In [ ]:
for i in range(10):
    fig, ax = ox.plot_graph(G, bgcolor='k', show=False, close=False, node_size=0)
    
    y, x = linestrings[i].xy
    ax.plot(x, y, color='red', linewidth=4)

    route1_list = route1['freeflow_route'].iloc[most_diff[i]]
    route2_list = route2['penalized_route'].iloc[most_diff[i]]
    route3_list = route3['penalized_route'].iloc[most_diff[i]]

    routes = [route1_list, route2_list, route3_list]
    route_colors = ['yellow', 'green', 'blue']

    ox.plot_graph_routes(G, routes=routes, route_colors=route_colors, route_linewidth=3, bgcolor='k', node_size=0, ax=ax)

plt.show()

In [ ]:
timecomparison.iloc[most_diff][0:10]

## Least

In [ ]:
diff = diff.sort_values(ascending=True, by='total')
less_diff = diff[diff['total'] > 0]
less_diff.head(20)

In [ ]:
od_data_less_index = less_diff.head(20).index
od_data_less_index

In [ ]:
od_data_least = od_data['polyline'][od_data_less_index]
linestrings = [LineString(ast.literal_eval(item)) for item in od_data_least]

In [ ]:
for i in range(10):
    fig, ax = ox.plot_graph(G, bgcolor='k', show=False, close=False, node_size=0)
    
    y, x = linestrings[i].xy
    ax.plot(x, y, color='red', linewidth=4)

    route1_list = route1['freeflow_route'].iloc[od_data_less_index[i]]
    route2_list = route2['penalized_route'].iloc[od_data_less_index[i]]
    route3_list = route3['penalized_route'].iloc[od_data_less_index[i]]

    routes = [route1_list, route2_list, route3_list]
    route_colors = ['yellow', 'green', 'blue']

    ox.plot_graph_routes(G, routes=routes, route_colors=route_colors, route_linewidth=3, bgcolor='k', node_size=0, ax=ax)

plt.show()

In [ ]:
timecomparison.iloc[od_data_less_index][0:10]

## Median

In [ ]:
diff = diff.sort_values(ascending=True, by='total')

median_index = len(diff) // 2

lower_bound = max(0, median_index - 5)
upper_bound = min(len(diff), median_index + 5)

median_10 = diff.iloc[lower_bound:upper_bound]

In [ ]:
m_index = median_10.index
median_route = od_data['polyline'][m_index]

In [ ]:
linestrings = [LineString(ast.literal_eval(item)) for item in median_route]

In [ ]:
for i in range(10):
    fig, ax = ox.plot_graph(G, bgcolor='k', show=False, close=False, node_size=0)
    
    y, x = linestrings[i].xy
    ax.plot(x, y, color='red', linewidth=4)

    route1_list = route1['freeflow_route'].iloc[m_index[i]]
    route2_list = route2['penalized_route'].iloc[m_index[i]]
    route3_list = route3['penalized_route'].iloc[m_index[i]]

    routes = [route1_list, route2_list, route3_list]
    route_colors = ['yellow', 'green', 'blue']

    ox.plot_graph_routes(G, routes=routes, route_colors=route_colors, route_linewidth=3, bgcolor='k', node_size=0, ax=ax)

plt.show()

In [ ]:
timecomparison.iloc[m_index][0:10]

## Negative Difference

In [ ]:
diff = diff.sort_values(ascending=True, by='total')
shorter_google = diff[diff['total'] < 0]
shorter_index = shorter_google.index

In [ ]:
shorter_index

In [ ]:
negative = od_data['polyline'][shorter_index]
linestrings = [LineString(ast.literal_eval(item)) for item in negative]

In [ ]:
for i in range(10):
    fig, ax = ox.plot_graph(G, bgcolor='k', show=False, close=False, node_size=0)
    
    y, x = linestrings[i].xy
    ax.plot(x, y, color='red', linewidth=4)

    route1_list = route1['freeflow_route'].iloc[shorter_index[i]]
    route2_list = route2['penalized_route'].iloc[shorter_index[i]]
    route3_list = route3['penalized_route'].iloc[shorter_index[i]]

    routes = [route1_list, route2_list, route3_list]
    route_colors = ['yellow', 'green', 'blue']

    ox.plot_graph_routes(G, routes=routes, route_colors=route_colors, route_linewidth=3, bgcolor='k', node_size=0, ax=ax)

plt.show()

In [ ]:
timecomparison.iloc[shorter_index][0:10]